### Generating synthetic training data for yolov5 model training

In [3]:
# Install yolov5 and its requirements, as well as our project's dependencies
![ ! -d "yolov5" ] && git clone https://github.com/ultralytics/yolov5
![ -d "yolov5" ] && cd yolov5 && git pull
!cd yolov5 && pip install -qr requirements.txt
!pip install -qr requirements.txt


Already up to date.


In [ ]:
#!rm -rf background_images
#!unzip "bg-*".zip -d background_images

In [4]:
from skimage.io import *
from skimage import data
from sklearn.model_selection import train_test_split
from pathlib import Path
from collections import namedtuple
from math import floor
from random import shuffle, randint
from itertools import combinations
from iteration_utilities import grouper
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2
import hashlib
import os
import io
import shutil
import glob
import math
%matplotlib inline

In [5]:
# source: https://stackoverflow.com/questions/40895785/using-opencv-to-overlay-transparent-image-onto-another-image
def add_transparent_image(background, foreground, x_offset=None, y_offset=None):
    bg_h, bg_w, bg_channels = background.shape
    fg_h, fg_w, fg_channels = foreground.shape

    assert bg_channels == 3, f'background image should have exactly 3 channels (RGB). found:{bg_channels}'
    assert fg_channels == 4, f'foreground image should have exactly 4 channels (RGBA). found:{fg_channels}'

    # center by default
    if x_offset is None: x_offset = (bg_w - fg_w) // 2
    if y_offset is None: y_offset = (bg_h - fg_h) // 2

    w = min(fg_w, bg_w, fg_w + x_offset, bg_w - x_offset)
    h = min(fg_h, bg_h, fg_h + y_offset, bg_h - y_offset)

    if w < 1 or h < 1: return

    # clip foreground and background images to the overlapping regions
    bg_x = max(0, x_offset)
    bg_y = max(0, y_offset)
    fg_x = max(0, x_offset * -1)
    fg_y = max(0, y_offset * -1)
    foreground = foreground[fg_y:fg_y + h, fg_x:fg_x + w]
    background_subsection = background[bg_y:bg_y + h, bg_x:bg_x + w]

    # separate alpha and color channels from the foreground image
    foreground_colors = foreground[:, :, :3]
    alpha_channel = foreground[:, :, 3] / 255  # 0-255 => 0.0-1.0

    # construct an alpha_mask that matches the image shape
    alpha_mask = np.dstack((alpha_channel, alpha_channel, alpha_channel))

    # combine the background with the overlay image weighted by alpha
    composite = background_subsection * (1 - alpha_mask) + foreground_colors * alpha_mask

    # overwrite the section of the background image that has been updated
    background[bg_y:bg_y + h, bg_x:bg_x + w] = composite

In [7]:
Item = namedtuple("Item", 'w h color')
Rectangle = namedtuple("Rectangle", "x y w h")
Border = namedtuple("Border", "w h")
Canvas = namedtuple("Canvas", "width height")

def overlap(rects):
    for rectA, rectB in list(combinations(rects, 2)):
        rectA_x2 = rectA.x + rectA.w
        rectA_y2 = rectA.y + rectA.h
        rectB_x2 = rectB.x + rectB.w
        rectB_y2 = rectB.y + rectB.h
        if rectA.x <= rectB_x2 and rectA_x2 >= rectB.x and rectA.y <= rectB_y2 and rectA_y2 >= rectB.y:
            print(f'A.x1 = {rectA.x}, A.x2 = {rectA_x2} / B.x1 = {rectB.x}, B.x2 = {rectB_x2}')
            print(f'A.y1 = {rectA.y}, A.y2 = {rectA_y2} / B.y1 = {rectB.y}, B.y2 = {rectB_y2}')
            return True
    return False
def out_of_area(canvas, rects):
    for rect in rects:
        if rect.x < 0 or (rect.x + rect.w) > canvas.width or rect.y > canvas.height or rect.y < 0:
            return True
    return False

def draw_items(background, labels, items, border):
    rects = []
    label = io.StringIO()
    max_h = max(items, key=lambda item: item[1].shape[0])[1].shape[0] + border.h
    max_w = max(items, key=lambda item: item[1].shape[1])[1].shape[1] + border.w  
    canvas = Canvas(height=background.shape[0], width=background.shape[1])
    # max_squares = min(floor(canvas.width / max_w), floor(canvas.height / max_h))
    max_squares = floor(canvas.width / max_w) * floor(canvas.height / max_h)
    assert(len(items) <= max_squares)
    grids = list(range(max_squares))
    shuffle(grids)
    for item in items:
        # canvas.fill_style = item.color
        pos = grids.pop()
        # item_x = (pos * max_w) + randint(0, max_w - item.w)
        item_x = (pos % floor(canvas.width / max_w) * max_w)
        item_x = randint(item_x, item_x + max_w - item[1].shape[1] - border.w)
        item_y = floor(pos / floor(canvas.width / max_w)) * max_h
        # print(f'before: item_y = {item_y}, pos = {pos}')
        item_y = randint(item_y, item_y + max_h - item[1].shape[0] - border.h)
        # print(f'after: {item_y}', item.h)
        # item_y = item_y + max_h
        rects.append(Rectangle(x = item_x, y = item_y, w = item[1].shape[1], h = item[1].shape[0]))
        # canvas.fill_rect(item_x, item_y, item.w, item.h)
        add_transparent_image(background, item[1], item_x, item_y)
        x_center = math.floor(((item_x + (item_x + item[1].shape[1])) / 2)) / background.shape[1]
        y_center = math.floor((item_y + (item_y + item[1].shape[0])) / 2) / background.shape[0]
        label.write(f'{item_names.index(item[0])} {x_center} {y_center} {item[1].shape[1] / background.shape[1]} {item[1].shape[0] / background.shape[0]}\n')
    labels.append(label)
    # assert(overlap(rects) == False)
    # assert(out_of_area(canvas, rects) == False)
def generate_images(item_images, background_files):
    generated_images = []
    generated_labels = []
    train_ratio = 0.70
    validation_ratio = 0.20
    test_ratio = 0.10
    hash = hashlib.md5()
    background_images = [(Path(name).stem, cv2.cvtColor(cv2.imread(name, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)) for name in background_files]
    # for item_name, item_image in item_images:
    for background_name, background_image in background_images:
        background = background_image.copy()
        try:
            draw_items(background, generated_labels, random.sample(
                item_images, randint(2, 5)), border=Border(10, 10))
            generated_images.append(background)
        except ValueError:
            break

    x_train, x_test = train_test_split(
        list(zip(generated_images, generated_labels)), test_size=(1 - train_ratio))
    x_val, x_test = train_test_split(
        x_test, test_size=test_ratio/(test_ratio + validation_ratio))

    for current_split, dataset in zip(['train', 'validate', 'test'], [x_train, x_val, x_test]):
        for img, label in dataset:
            # cv2.rectangle(img, (x_offset, y_offset), (x_offset + w, y_offset + h), (0,255,0), 2)
            # imshow(img)
            hash.update(repr(img).encode('utf-8'))
            cv2.imwrite(
                f'generated_images/{current_split}/images/{hash.hexdigest()}.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
            with open(f'generated_images/{current_split}/labels/{hash.hexdigest()}.txt', 'w') as f:
                print(label.getvalue(), file=f)
        pass


In [8]:

item_names = [Path(name).stem for name in glob.glob('individual_items/*.png')]
item_images = [(Path(name).stem, cv2.cvtColor(cv2.imread(name, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGBA)) for name in glob.glob('individual_items/*.png')]
background_files = np.random.choice(glob.glob('background_images/train/*.jpg'), size=500, replace=True)
try:
    shutil.rmtree('generated_images')
except:
    pass
finally:
    dirs = ['generated_images/train/images', 'generated_images/train/labels', 'generated_images/validate/images',
            'generated_images/validate/labels', 'generated_images/test/images', 'generated_images/test/labels']
    [os.makedirs(path, exist_ok=True) for path in dirs]

for batch in grouper(background_files, 100):
    generate_images(item_images, batch)

with open('data.yaml', 'w') as f:
    f.write(f'train: ../generated_images/train/images\n')
    f.write(f'val: ../generated_images/validate/images\n')
    f.write(f'nc: {len(item_names)}\n')
    f.write('names: [')
    for name in item_names[:-1]:
        f.write(f'\'{name}\',')
    f.write(f'\'{item_names[-1]}\']')

### Training the model

In [ ]:
!python yolov5/train.py --weights yolov5s.pt --data data.yaml --batch 64 --img 640 --epochs 500